In [36]:
%load_ext autoreload
%autoreload 2

from cellmates.data.breast import BreastCancerTissueDataset
from cellmates.data.dataset import CellMatesDataset
from cellmates.utils import MAX_EFFECTIVE_DISTANCE

import torch
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
responder_cell_type = 'F'


dss = [BreastCancerTissueDataset(
    tissue_idx=i, 
    effective_distance=MAX_EFFECTIVE_DISTANCE, 
    responder_cell_type=responder_cell_type)
    for i in range(1,4)
]

In [31]:
ds = dss[0]
samples = [ds[3], ds[1]]
ds = CellMatesDataset(samples)

In [63]:
B = 2
L = 3
D = 4
padding_mask_BL = torch.ones((B,L))
padding_mask_BL[0,2] = 0
hidden_BLD = torch.ones((B,L,D))

torch.einsum("BLD,BL->BD", hidden_BLD, padding_mask_BL)

tensor([[2., 2., 2., 2.],
        [3., 3., 3., 3.]])

In [68]:
padding_mask_BL

tensor([[1., 1., 0.],
        [1., 1., 1.]])

In [81]:
padding_mask_BL[0].repeat(3,1) * padding_mask_BL[0].repeat(3,1).T

tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 0.]])

In [101]:
_ = padding_mask_BL.repeat(1,H,1,L).reshape((B,H,L,L))
padding_mask_BHLL = _ * _.permute(0,1,3,2)
padding_mask_BHLL

tensor([[[[1., 1., 0.],
          [1., 1., 0.],
          [0., 0., 0.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 0.],
          [1., 1., 0.],
          [0., 0., 0.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 0.],
          [1., 1., 0.],
          [0., 0., 0.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 0.],
          [1., 1., 0.],
          [0., 0., 0.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [137]:
from cellmates.data.dataset import collate_fn

dl = DataLoader(
    ds, batch_size=2, collate_fn=collate_fn, shuffle=True
)

In [138]:
batch = next(iter(dl))

In [139]:
from cellmates.model.transformer import CellMatesTransformer

In [145]:
CellMatesTransformer(
    D = (16 * 4),
    K=4
).forward(batch['cell_types_BL'], batch['distances_BLL'], batch['padding_mask_BL'])

tensor([[38.1759],
        [36.9896]], grad_fn=<AddmmBackward0>)

In [119]:
import numpy as np

np.prod([2, 191, 16, 512])

3129344